In [ ]:
### dataset 폴더 내 TL_multiple_choice.json

In [ ]:
!pip install datasets==2.21.0

In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset
import huggingface_hub

### JSON 형태의 행정 문서 대상 기계독해 데이터 (출처: AI허브 -> 한국어/텍스트 조회 -> "한국어 지식기반 관계데이터")

In [ ]:
# https://drive.google.com/uc?export=download&id=파일ID
url = "https://drive.google.com/uc?export=download&id=1JehPjKhyw9NlVdoKBv7ZtxZLqaEjN7H9"
response = requests.get(url)

original_data = json.loads(response.text)
print(original_data)

Buffered data was truncated after reaching the output size limit.

In [ ]:
# 원본데이터의 키 확인하기
original_data.keys()

dict_keys(['Dataset', 'data'])

In [ ]:
len( original_data["data"] )

15085

In [ ]:
datas = original_data["data"]

### 필요 데이터를 alpaca 포맷 (instruction, input, ouput) 형태로 수집

In [ ]:
# 각 데이터를 하나씩 끊어서 보기
eachData = datas[0]

In [ ]:
# 하나의 데이터 셋 내 주요 파라그래프 키 확인하기
eachData.keys()

dict_keys(['doc_id', 'doc_title', 'doc_source', 'doc_published', 'doc_class', 'created', 'paragraphs'])

In [ ]:
# 하나의 데이터 셋 필요 파라그래프 하나씩 끊어서 보기 # 어려우면 여기 반복문은 돌리지 말기
eachParagraphs = eachData["paragraphs"][0]

In [ ]:
# 딕셔너리 구조의 키 확인하기
eachParagraphs.keys()

dict_keys(['context', 'qas', 'context_id'])

In [ ]:
# 딕셔너리 구조의 키 중 주요 정보 키값으로 가져오기
qasData = eachParagraphs["qas"]

In [ ]:
# 하나의 Q&A 질의문 끊어서 보기
eachQas = qasData[0]

In [ ]:
# 보물1: 질문 para -> qas -> question
eachQas["question"]

'센터장은 아동학대예방을 위해 누구에게 교육이 필요한지 위원들에게 전달했니'

In [ ]:
# 보물2: 제시문1 para -> qas -> answers -> clue_text
eachQas["answers"]["clue_text"]

'센터장의 설명이 끝난 후에는 센터장 및 강북구청 관계자와 특위 위원들 간의 질의 응답시간이 마련됐다.\n특위 위원들은 “아동학대예방을 위해서는 부모교육과 보육교사들에 대한 더욱 철저한 교육이 필요하다”며, “육아종합지원센터 뿐만 아니라 다른 기관 들과의 연계를 통해 아동학대예방에 위해 더욱 힘써 달라”고 당부했다.'

In [ ]:
# 보물3: 제시문2 para -> qas -> answers -> options
eachQas["answers"]["options"]

['부모', '지역 주민', '센터장', '자녀']

In [ ]:
# 보물4: 정답 para -> qas -> answers -> text
eachQas["answers"]["text"]

'부모'

In [ ]:
### 다 풀고나면? 아래와 같이 정리되면 됨
### instrunction : 보물1 (질문)
### input        : 보물2 (제시문1) + \n options \n + 보물3 (제시문2)
### output       : 보물4 (정답)

In [ ]:
## 1 data -> n paragraph
## 1 paragraph -> n qas
## 1 qas -> 1 answer, 1 text, n options

In [ ]:
alpaca_data = []

for data in datas :
    for para in data["paragraphs"]:
        for qas in para["qas"]:
            instruction = qas["question"]
            input = qas["answers"]["clue_text"] + "\n options \n" + ", ".join(qas["answers"]["options"])
            # print(input)
            output = qas["answers"]["text"]
            alpaca_data.append([instruction, input, output])

alpaca_data = pd.DataFrame(alpaca_data, columns=["instruction", "input", "output"])
alpaca_data

,instruction,input,output
0,센터장은 아동학대예방을 위해 누구에게 교육이 필요한지 위원들에게 전달했니,센터장의 설명이 끝난 후에는 센터장 및 강북구청 관계자와 특위 위원들 간의 질의 응...,부모
1,연구 성과 및 정보를 제공하는 보건환경톡톡의 7편의 내용은 무엇이야,"7편 하수찌꺼기 자원화, 8편 전통장류 품질 및 안전성, 9편 슈퍼푸드 잔류농약, ...",하수찌꺼기 자원화
2,무엇이 연구 실적 및 정보를 확인할 수 있는 보건환경톡톡의 7편의 내용일까,"7편 하수찌꺼기 자원화, 8편 전통장류 품질 및 안전성, 9편 슈퍼푸드 잔류농약, ...",하수찌꺼기 자원화
3,어떤 수단을 활용해 2021 제1차 강북구 치매안심센터 자문위원회 회의가 열릴 예정이니,건 명: 2021년 제1차 강북구 치매안심센터 자문위원회 서면개최\n options...,문서
4,한옥 등 건축자산의 진흥에 관한 법률 중 제8조에 해당하는 조례가 뭐야,❍ 서울특별시 각종 위원회의 설치·운영에 관한 조례 제8조(위원회의 구성)\n ...,서울특별시 각종 위원회의 설치·운영에 관한 조례
...,...,...,...
25161,"어떤 철새가 흑산도에서 11월 12월 두 달 동안 6,000마리 이상 발견됐지",하지만 11~12월 두 달 동안 어디에 머무는지 제대로 알려지지 않았는데 이번 조사...,괭이갈매기
25162,친환경 순환경제 정착을 위해 어떤 제도를 도입할 계획이야,"또한, 친환경 순환경제 정착을 위해 자원순환 성과관리제와 폐기물 처분분담금, 재활용...",재활용 네거티브제
25163,어떤 이행과제로 친환경 순환경제 정착을 촉진할 계획이지,"또한, 친환경 순환경제 정착을 위해 자원순환 성과관리제와 폐기물 처분분담금, 재활용...",재활용 네거티브제
25164,에너지 수요관리 강화를 위해 대한민국 정부가 추진하는 과제는 무엇이야,"(에너지 수요관리 강화) 수요관리형 전기요금제 확대, 에너지 수요관리 핵심기술 확보...",수요관리형 전기요금제 확대


In [ ]:
# pd.DataFrame( zip(instructionList, inputList, outputList), columns=["instruction", "input", "output"] )
# pd.DataFrame( alpaca_data )

,instruction,input,output
0,센터장은 아동학대예방을 위해 누구에게 교육이 필요한지 위원들에게 전달했니,센터장의 설명이 끝난 후에는 센터장 및 강북구청 관계자와 특위 위원들 간의 질의 응...,부모
1,연구 성과 및 정보를 제공하는 보건환경톡톡의 7편의 내용은 무엇이야,"7편 하수찌꺼기 자원화, 8편 전통장류 품질 및 안전성, 9편 슈퍼푸드 잔류농약, ...",하수찌꺼기 자원화
2,무엇이 연구 실적 및 정보를 확인할 수 있는 보건환경톡톡의 7편의 내용일까,"7편 하수찌꺼기 자원화, 8편 전통장류 품질 및 안전성, 9편 슈퍼푸드 잔류농약, ...",하수찌꺼기 자원화
3,어떤 수단을 활용해 2021 제1차 강북구 치매안심센터 자문위원회 회의가 열릴 예정이니,건 명: 2021년 제1차 강북구 치매안심센터 자문위원회 서면개최\n options...,문서
4,한옥 등 건축자산의 진흥에 관한 법률 중 제8조에 해당하는 조례가 뭐야,❍ 서울특별시 각종 위원회의 설치·운영에 관한 조례 제8조(위원회의 구성)\n ...,서울특별시 각종 위원회의 설치·운영에 관한 조례
...,...,...,...
25161,"어떤 철새가 흑산도에서 11월 12월 두 달 동안 6,000마리 이상 발견됐지",하지만 11~12월 두 달 동안 어디에 머무는지 제대로 알려지지 않았는데 이번 조사...,괭이갈매기
25162,친환경 순환경제 정착을 위해 어떤 제도를 도입할 계획이야,"또한, 친환경 순환경제 정착을 위해 자원순환 성과관리제와 폐기물 처분분담금, 재활용...",재활용 네거티브제
25163,어떤 이행과제로 친환경 순환경제 정착을 촉진할 계획이지,"또한, 친환경 순환경제 정착을 위해 자원순환 성과관리제와 폐기물 처분분담금, 재활용...",재활용 네거티브제
25164,에너지 수요관리 강화를 위해 대한민국 정부가 추진하는 과제는 무엇이야,"(에너지 수요관리 강화) 수요관리형 전기요금제 확대, 에너지 수요관리 핵심기술 확보...",수요관리형 전기요금제 확대


In [ ]:
### 아래는 참고용 코드 (허깅페이스에 LLM 파인튜닝용 데이터 셋 저장)

In [ ]:
# 토큰>

In [ ]:
# # Dataset 형태로 변환
dataset = Dataset.from_pandas( alpaca_data )

# # 허깅페이스 로그인 ( https://huggingface.co/ 회원가입 후 DATASET 하나 추가 이후 API 키 생성)
huggingface_hub.login()

In [ ]:
# # 허깅페이스에 업로드 ( 본인의 데이터셋 생성한 주소를 적어야함)
dataset.push_to_hub("seohyunee22/gigedokhe")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/seohyunee22/gigedokhe/commit/75320b55bbd7c74af239eea865c9586710b1007b', commit_message='Upload dataset', commit_description='', oid='75320b55bbd7c74af239eea865c9586710b1007b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/seohyunee22/gigedokhe', endpoint='https://huggingface.co', repo_type='dataset', repo_id='seohyunee22/gigedokhe'), pr_revision=None, pr_num=None)